In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
import datetime

In [3]:
import tensorflow as tf

'2.1.0'

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

Using TensorFlow backend.


In [5]:
df=pd.read_csv(r'/kaggle/input/raw-4-years-data-for-corpus/reddit_data_raw_4years.csv')

In [6]:
corpus=[]

for title in df['title']:
    corpus.append(title)
    
len(corpus)

433802

In [7]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    try:
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
        doc = doc.lower()
        doc = doc.strip()
        # tokenize document
        tokens = wpt.tokenize(doc)
        # filter stopwords out of document
        filtered_tokens = [token for token in tokens if token not in stop_words]
        # re-create document from filtered tokens
        doc = ' '.join(filtered_tokens)
        return doc
    except:
        return ""

normalize_corpus = np.vectorize(normalize_document)

In [ ]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

In [ ]:
MAX_NB_WORDS = 30000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 30
# This is fixed.
EMBEDDING_DIM = 300

In [ ]:
from gensim.models import word2vec

# tokenize sentences in corpus
wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in norm_corpus]

# Set values for various parameters
feature_size = EMBEDDING_DIM    # Word vector dimensionality  
window_context = 6          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count=min_word_count,
                          sample=sample, iter=50)

In [ ]:
words = w2v_model.wv.index2word
wvs = w2v_model.wv[words]

In [ ]:
df_train=pd.read_csv('/kaggle/input/reddit-data-balanced/reddit_data_balanced.csv')

In [ ]:
df_train.head(30)

In [ ]:
df_train=df_train.loc[df_train['flair']!='other']

In [ ]:
df_train.shape

In [ ]:
corpus2=[]

for title in df_train['title']:
    corpus2.append(title)
    
len(corpus2)


In [ ]:
norm_corpus2 = normalize_corpus(corpus2)
norm_corpus2


In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(norm_corpus2)


In [ ]:
list_tokenized_train = tokenizer.texts_to_sequences(norm_corpus2)


In [ ]:
X = pad_sequences(list_tokenized_train, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
print('Found %s unique tokens.' % len(tokenizer.word_index))

In [ ]:
t_dict={}

for word in w2v_model.wv.vocab:
    t_dict[word]=w2v_model[word]

In [ ]:
# t_dict

In [ ]:
all_embs = np.stack(t_dict.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [ ]:
word_index = tokenizer.word_index
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS: continue
    embedding_vector = t_dict.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [ ]:
embedding_matrix[0]

In [ ]:
df_train.head()

In [ ]:
df_train['category_id'] = df_train['flair'].factorize()[0]
category_id_df = df_train[['flair', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'flair']].values)
df_train.head()


In [ ]:
labels=np.reshape(np.array(df_train['category_id']), (83000, 1))
from keras.utils import to_categorical
y_binary = to_categorical(labels)

# labels2=np.reshape(np.array(df_train2['category_id']), (118000, 1))
# from keras.utils import to_categorical
# y_binary2 = to_categorical(labels2)

In [ ]:
embedding_matrix.shape

In [ ]:
y_binary.shape

In [ ]:
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

# model = Sequential()
# # x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix]))
# # model.add(SpatialDropout1D(0.2))
# model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(14, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Activation
from keras import optimizers

model = Sequential()
# x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(1000, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_initializer='glorot_uniform'))
# model.add(GlobalAveragePooling1D())
# model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
# from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
# from keras.layers.normalization import BatchNormalization
# from keras.layers import Dense, Activation

# model = Sequential()
# # x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix]))
# model.add(Conv1D(filters=64, kernel_size=3 ,strides=1, padding='same' , activation= 'relu')) 
# model.add(MaxPooling1D(pool_size=2))
# model.add(SpatialDropout1D(0.4))
# model.add(LSTM(1000, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_initializer='glorot_uniform'))
# # model.add(GlobalAveragePooling1D())
# # model.add(BatchNormalization())
# model.add(GlobalMaxPooling1D())
# model.add(Dense(14, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print(model.summary())

In [ ]:
# from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
# from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
# from keras.layers.normalization import BatchNormalization
# from keras.layers import Dense, Activation
# from keras.models import Model


# inp = Input( shape=(30,))
# x=Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix])(inp)
# x=SpatialDropout1D(0.2)(x)
# x=LSTM(1000, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(x)
# x=BatchNormalization()(x)
# x=Activation('relu')(x)
# avg_pool = GlobalAveragePooling1D()(x)
# max_pool = GlobalMaxPooling1D()(x)
# conc = concatenate([avg_pool, max_pool])
# out=Dense(14, activation='softmax')(conc)

# model = Model(inputs=inp, outputs=out)
# model.compile(loss='logcosh', optimizer='adam', metrics=['accuracy'])

In [ ]:
# from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D

# model = Sequential()
# # x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, weights=[embedding_matrix]))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(1000, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(14, activation='softmax'))
# model.compile(loss=tf.keras.losses.Huber(delta=1.0), optimizer='adam', metrics=['accuracy'])
# print(model.summary())

In [ ]:
X.shape

In [ ]:
epochs = 10
batch_size = 64

history = model.fit(X[0:70000], y_binary[0:70000],
                    batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
df_train.head()

In [ ]:
y_pred=model.predict(X[70000:])

In [ ]:
classification_report(df_train['category_id'][70000:], np.argmax(y_pred, axis=1), output_dict=True)

In [ ]:
model.save('/kaggle/working/reddit_predictor.h5')

In [ ]:
model_json = model.to_json()
with open("/kaggle/working/model.json", "w") as json_file:
    json_file.write(model_json)